In [20]:
import netgen.geom2d as geom2d
from ngsolve import *
from ngsolve.webgui import Draw


geo = geom2d.SplineGeometry()
pnums = [ geo.AddPoint (x,y,maxh=0.01) for x,y in [(0,0), (1,0), (1,0.1), (0,0.1)] ]
for p1,p2,bc in [(0,1,"bot"), (1,2,"right"), (2,3,"top"), (3,0,"left")]:
     geo.Append(["line", pnums[p1], pnums[p2]], bc=bc)
mesh = Mesh(geo.GenerateMesh(maxh=0.01))

In [21]:
#

# geometric non-linear elasticity with Neo-Hooke hyperelastic material
#
# featuring automatic differentiation in SymbolicEnergy
#

E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

fes = H1(mesh, order=2, dirichlet="left", dim=mesh.dim)
# fes = VectorH1(mesh, order=2, dirichlet="left")

u  = fes.TrialFunction()

force = CoefficientFunction( (0,1/2) )

I = Id(mesh.dim)
F = I + Grad(u)
C = F.trans * F
E = 0.5 * (C-I)

def Pow(a, b):
    return a**b  # exp (log(a)*b)
  
def NeoHooke (C):
    return 0.5 * mu * (Trace(C-I) + 2*mu/lam * Pow(Det(C),-lam/2/mu) - 1)



factor = Parameter(1)

a = BilinearForm(fes, symmetric=False)
a += Variation (NeoHooke(C).Compile()*dx)
a += Variation ((-factor * InnerProduct(force,u) ).Compile()*dx)


u = GridFunction(fes)
u.vec[:] = 0

res = u.vec.CreateVector()
w = u.vec.CreateVector()


for loadstep in range(10):
    
    print ("loadstep", loadstep)
    factor.Set (loadstep+1)
    
    for it in range(5):
        print ("Newton iteration", it)
        print()
        print ("energy = ", a.Energy(u.vec))
        a.Apply(u.vec, res)
        a.AssembleLinearization(u.vec)
        inv = a.mat.Inverse(fes.FreeDofs() ) 
        w.data = inv*res
        print ("err^2 = ", InnerProduct (w,res))
        u.vec.data -= w


loadstep 0
Newton iteration 0

energy =  8.750000000000004
err^2 =  0.006956119577603163
Newton iteration 1

energy =  8.893483845033886
err^2 =  0.3124310377917503
Newton iteration 2

energy =  8.748200086061416
err^2 =  0.0030799214338498642
Newton iteration 3

energy =  8.746675673924322
err^2 =  4.57003284163197e-05
Newton iteration 4

energy =  8.74665277507665
err^2 =  2.5459537927568226e-07
loadstep 1
Newton iteration 0

energy =  8.740196699501944
err^2 =  0.00558690185604044
Newton iteration 1

energy =  8.812485812724944
err^2 =  0.15621138164658382
Newton iteration 2

energy =  8.73819516269208
err^2 =  0.000781033458691565
Newton iteration 3

energy =  8.737824566092147
err^2 =  0.00011449833596381448
Newton iteration 4

energy =  8.737791812309965
err^2 =  6.127134478684082e-05
loadstep 2
Newton iteration 0

energy =  8.726744895460724
err^2 =  0.0034888122869844705
Newton iteration 1

energy =  8.742129597604425
err^2 =  0.03443010649488529
Newton iteration 2

energy =  8

In [22]:

E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

Pspace = L2(mesh, order=2) 
Uspace = VectorH1(mesh,order=3, dirichlet="left") 
Pispace = MatrixValued(L2(mesh,order = 2), symmetric=True)
 

fes = FESpace([Pispace, Uspace, Pspace])
PI,U,P  = fes.TrialFunction()

TAU, V, Q = fes.TestFunction()

force = CoefficientFunction( (0,0.5) )

I = Id(mesh.dim)
F = I + Grad(U)
C = F.trans * F
E = 0.5 * (C-I)

def Pow(a, b):
    return a**b  # exp (log(a)*b)
def Diag(F):
    return CF((F[0,0],0,0,F[1,1]), dims = (2,2))
  
def Neo_Dev(F):
    return 0.5 * mu * (2*Diag(F) - (Det(F.trans * F)**(-lam/(2*mu))) * Cof(F.trans * F).trans * (2 * Diag(F)))



factor = Parameter(1)

a = BilinearForm(fes)
a += InnerProduct(PI - (Neo_Dev(F)*(Id(mesh.dim) + Grad(U)).trans - P * Id(mesh.dim)), TAU)*dx
a += InnerProduct(PI*Inv(Id(mesh.dim) + Grad(U)).trans, Grad(V)) * dx
a += InnerProduct((Det(Grad(U)+ Id(mesh.dim)) - 1) ,Q) * dx

a += -factor * InnerProduct(force,V)*dx


gfu = GridFunction(fes)
gfu.vec[:] = 0

res = gfu.vec.CreateVector()
w = gfu.vec.CreateVector()

damp = 1
plus_load = 1
# plus_load * many_loads = 10
for loadstep in range(int(10/plus_load)):
    
    print ("loadstep", loadstep)
    factor.Set (loadstep + plus_load)
    
    for it in range(5):
        print ("Newton iteration", it)
        print ("energy = ", a.Energy(gfu.vec))
        a.Apply(gfu.vec, res)
        a.AssembleLinearization(gfu.vec)
        inv = a.mat.Inverse(fes.FreeDofs()) 
        w.data = inv*res
        print ("err^2 = ", InnerProduct (w,res))
        gfu.vec.data -= damp * w




loadstep 0
Newton iteration 0
energy =  0.0
err^2 =  0.004389330345747958
Newton iteration 1
energy =  -0.000137142628943581
err^2 =  -0.006505933573125255
Newton iteration 2
energy =  0.00038868235682729666
err^2 =  3.0498653040489223e-05
Newton iteration 3
energy =  3.1433421176497e-07
err^2 =  -7.858977554312497e-08
Newton iteration 4
energy =  2.0402997727635094e-08
err^2 =  -3.72345084896591e-10
loadstep 1
Newton iteration 0
energy =  -0.002079582531111047
err^2 =  0.003921187211661831
Newton iteration 1
energy =  0.010789447551728934
err^2 =  -0.002404470190348365
Newton iteration 2
energy =  0.00013263299679450798
err^2 =  6.531213223729048e-06
Newton iteration 3
energy =  7.6671457970518e-07
err^2 =  2.8035009668785146e-12
Newton iteration 4
energy =  2.577877886894201e-13
err^2 =  -1.1288516819712653e-24
loadstep 2
Newton iteration 0
energy =  -0.003850449504755812
err^2 =  0.003135571961117278
Newton iteration 1
energy =  0.021157767598129595
err^2 =  0.00030608255997904014
N

In [23]:
Draw (gfu.components[1], mesh, "disp")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [24]:
Draw(u - gfu.components[1],mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene